It seems that differences in output to the original examples.mol2 file have to do with file conversions or my supplier function. 

We would likely be best suited by using native SDF.

However, my conversion tools (e.g. OpenBabel) did not work in terms of reproducing the original results when using my SDF file supplier method. 

Francesco found that conversion via OpenBabel to SDF and then conversion back to mol2 via MOE did get very close to the original output. This solves the issue of needing an SDF supplier method (in case the error is on my end), as we could use this tool to convert our docking output sdf to mol2, get the desired data, and then continue. The differences seem to be in relatively minor changes in actually 3D atomic positions, but I not conclusively sure. It is somewhat tough though, because we ultimately do not have an example of a "correct" sdf. If I could successfully reproduce the mol2 results via conversion to sdf, it would solve a lot of headaches. There is also the necessary benchmarking we will have to do, so it is not a lost cause if we can not directly replicate the results.

Still, it may be most optimal to use a conversion tool that gives the demonstrated results.

Outside of OpenBabel, we could try other conversion tools. Like Schrodinger. 

In [1]:
#Used Maestro 2023-3 Structure Conversion to go from example.mol2 to schrodinger_convert.sdf 
%run refactor_Torsion_Strain.py -i schrodinger_convert.sdf  -o schrodinger_sdf.csv 

1 molecules finished reading. Calculating strain energy...
1 successful / 0 NA. Please check: schrodinger_sdf.csv


[15:13:44] Explicit valence for atom # 24 N, 4, is greater than permitted
[15:13:44] ERROR: Could not sanitize molecule ending on line 111
[15:13:44] ERROR: Explicit valence for atom # 24 N, 4, is greater than permitted
[15:13:44] Explicit valence for atom # 9 N, 4, is greater than permitted
[15:13:44] ERROR: Could not sanitize molecule ending on line 213
[15:13:44] ERROR: Explicit valence for atom # 9 N, 4, is greater than permitted


Unfortunately, this doesn't work. It is giving the exact error we might expect, the valences in the nitro groups. This might be due to my supplier function.

His supplier function works like so: 

```py 

m = Chem.rdmolfiles.MolFromMol2Block(block, sanitize=False, removeHs=False)

```

Mine uses rdkit's native SDF tool. I have sanitize as "True" because I had issues getting the OpenBabel based mol2 -> sdf conversion to then accept these compounds. Maybe I can try turning that off? I added functionality for that. Let's try. 

In [4]:
import importlib
import refactor_TL_Functions as rtf

importlib.reload(rtf)

%run refactor_Torsion_Strain.py -i schrodinger_convert.sdf  -o schrodinger_no_sanitize_sdf.csv 

3 molecules finished reading. Calculating strain energy...
0 successful / 3 NA. Please check: schrodinger_no_sanitize_sdf.csv


[15:31:21] 

****
Pre-condition Violation
RingInfo not initialized
Violation occurred on line 129 in file /Users/runner/miniforge3/conda-bld/rdkit_1678104767387/work/Code/GraphMol/RingInfo.cpp
Failed Expression: df_init
****

[15:31:21] 

****
Pre-condition Violation
RingInfo not initialized
Violation occurred on line 129 in file /Users/runner/miniforge3/conda-bld/rdkit_1678104767387/work/Code/GraphMol/RingInfo.cpp
Failed Expression: df_init
****

[15:31:21] 

****
Pre-condition Violation
RingInfo not initialized
Violation occurred on line 129 in file /Users/runner/miniforge3/conda-bld/rdkit_1678104767387/work/Code/GraphMol/RingInfo.cpp
Failed Expression: df_init
****



No, now I can't actually initalize the the mol objects. Sanitize needs to be True in sdf mol supplier. Can it be tweaked further? Can I use a similar from block strategy? What does the block look like? 


In [11]:
def Mol2MolSupplier(file=None):
    names = []
    mols = {}
    blocks = []
    with open(file, "r") as f:
        fileend = os.fstat(f.fileno()).st_size
        count = 0
        line = f.readline()
        while not f.tell() == fileend:
            if line.startswith("#") or line == "\n":
                line = f.readline()
            if line.startswith("@<TRIPOS>MOLECULE"):
                count += 1
                mol = []
                mol.append(line)
                line = f.readline()
                if line != "\n" and line.split()[0].strip() not in names:
                    name = line.split()[0].strip()
                else:
                    name = "mol2Number" + str(count)
                while not line.startswith("@<TRIPOS>MOLECULE"):
                    mol.append(line)
                    line = f.readline()
                    if f.tell() == fileend:
                        mol.append(line)
                        break
                block = ",".join(mol).replace(",", "")
                blocks.append(block)
                m = Chem.rdmolfiles.MolFromMol2Block(
                    block, sanitize=False, removeHs=False
                )
                names.append(name)
                mols[name] = m
    return (names, mols, blocks)

In [14]:
from rdkit import Chem
mol2_names, mol2_mols, mol2_blocks = Mol2MolSupplier("example.mol2")
# make variable one_mol that is equal to the first value of the first key in mol2_mols, expand to two and three
one_mol_mol2 = mol2_mols[list(mol2_mols.keys())[0]]
two_mol_mol2 = mol2_mols[list(mol2_mols.keys())[1]]
three_mol_mol2 = mol2_mols[list(mol2_mols.keys())[2]]

one_mol2_block = mol2_blocks[0]
two_mol2_block = mol2_blocks[1]
three_mol2_block = mol2_blocks[2]

In [16]:
print(one_mol2_block)

@<TRIPOS>MOLECULE
 ZINC000191583186      none
   52    53     0     0     0



@<TRIPOS>ATOM
      1 C1        -15.2579    21.1839   -23.5457 C.3        1  LIG1  -0.1300
      2 C2        -15.4512    20.0791   -22.5392 C.ar       1  LIG1  -0.0700
      3 C3        -16.1319    18.9302   -22.8964 C.ar       1  LIG1  -0.1200
      4 C4        -16.3092    17.9166   -21.9733 C.ar       1  LIG1  -0.1200
      5 C5        -15.8055    18.0519   -20.6930 C.ar       1  LIG1  -0.1300
      6 C6        -15.9994    16.9473   -19.6865 C.3        1  LIG1  -0.0900
      7 C7        -17.3043    17.1775   -18.9214 C.3        1  LIG1   0.0300
      8 H1        -17.2581    18.1361   -18.4046 H          1  LIG1   0.1500
      9 C8        -18.4771    17.1840   -19.9039 C.3        1  LIG1   0.0500
     10 O1        -18.5377    15.9273   -20.5816 O.3        1  LIG1  -0.5700
     11 N1        -17.4932    16.1008   -17.9403 N.4        1  LIG1  -0.5000
     12 H2        -18.4657    16.0542   -17.6750 H          